In [ ]:
import boto3
import botocore


sns = boto3.client('sns')
sqs = boto3.client('sqs')

In [ ]:
import os
account_no = os.environ['ACCOUNT_NO']

In [ ]:
failureNotificationRoleArn = 'arn:aws:iam::{}:role/SNSFailureFeedback'.format(account_no)

In [ ]:
ct = sns.create_topic(
    Name='topicOne',
    Attributes={
        'SQSFailureFeedbackRoleArn': failureNotificationRoleArn
    }
)
print(ct)
topic_arn1 = ct['TopicArn']

In [ ]:
ct = sns.create_topic(
    Name='topicTwo',
    Attributes={
        'SQSFailureFeedbackRoleArn': failureNotificationRolArn
    }
)
print(ct)
topic_arn2 = ct['TopicArn']

In [ ]:
cq = sqs.create_queue(
    QueueName='sampleQueue'
)
print(cq)
queue_url = cq['QueueUrl']
queue_arn = 'arn:aws:sqs:us-east-1:' + account_no + ':sampleQueue'

In [ ]:
sns.subscribe(
    TopicArn=topic_arn1,
    Protocol='sqs',
    Endpoint=queue_arn
)

In [ ]:
sns.subscribe(
    TopicArn=topic_arn2,
    Protocol='sqs',
    Endpoint=queue_arn
)

In [ ]:
def allow_sns_to_write_to_sqs(topicarn, queuearn):
    policy_document = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnLike":{{
          "aws:SourceArn": "{}noworky"
        }}
      }}
    }}
  ]
}}""".format(queuearn, topicarn)

    return policy_document

In [ ]:
#policy_json = allow_sns_to_write_to_sqs(topic_arn1, queue_arn)
policy_json = allow_sns_to_write_to_sqs('arn:aws:sns:us-east-1:' + account_no + ':*', queue_arn)

response = sqs.set_queue_attributes(
    QueueUrl = queue_url,
    Attributes = {
        'Policy' : policy_json
    }
)
print(response)

In [ ]:
sns.publish(
    TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicOne',
    Message='TOPIC ONE ONE ONE ONE ONE ONE ONE'
)

In [ ]:
sns.publish(
    TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicTwo',
    Message='TOPIC TWO TOPIC TWO'
)

In [ ]:
qurl = 'https://queue.amazonaws.com/' + account_no + '/sampleQueue'
rr = sqs.receive_message(
    QueueUrl=qurl
)
print(rr)

In [ ]:
if 'Messages' in rr:
    msg = rr['Messages'][0]
    rhandle = msg['ReceiptHandle']

    dm = sqs.delete_message(
        QueueUrl=qurl,
        ReceiptHandle=rhandle
    )

    print(dm)
else:
    print('no message received')

In [ ]:
client = boto3.client('logs')

In [ ]:
import time
nowish = int(time.time())
hour_ago = nowish - (60*60)

q = client.start_query(
    logGroupName='sns/us-east-1/427848627088/topicOne/Failure',
    startTime=hour_ago,
    endTime=nowish,
    queryString='fields @timestamp, @message | sort @timestamp desc | limit 20',
    limit=123
)

print(q)

In [ ]:
r = client.get_query_results(queryId=q['queryId'])
print(r)

### Clean Up

In [ ]:
try:
    sqs.delete_queue(
        QueueUrl='https://queue.amazonaws.com/' + account_no + '/sampleQueue'
    )
    print('queue deleted')
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'AWS.SimpleQueueService.NonExistentQueue':
        print('queue deleted')
    else:
        raise error

In [ ]:
try:
    sns.get_topic_attributes(TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicOne')
    sns.delete_topic(
        TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicOne'
    )
    print('topic deleted')
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'NotFound':
        print('topic already deleted')
    else:
        raise error

In [ ]:
import botocore
try:
    sns.get_topic_attributes(TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicTwo')
    sns.delete_topic(
        TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicTwo'
    )
    print('topic deleted')
except botocore.exceptions.ClientError as error:
    if error.response['Error']['Code'] == 'NotFound':
        print('topic already deleted')
    else:
        raise error